# Neural Network on MNIST Dataset
This Jupyter notebook implements a two-layer neural network and details the underlying mathematical concepts. The network is trained through the use of loss functions, gradients, and optimizers. The performance of the network is tested on the MNIST dataset. 

The implementation of the neural network uses the functionality of Numpy (http://www.numpy.org/) and Matplotlib (https://matplotlib.org/). Before running this notebook, make sure you have installed all of these packages in your local Jupyter instance.


In [ ]:
# Numpy and Matplotlib imports
import numpy as np
import matplotlib.pyplot as plt
import math

---

## Multiclass Classification

Our goal is to build a classifier to separate all 10 of the digits (classes/labels). 

We use multinomal logistic regression (aka softmax regression) where we define the posterior probability of label $y \in \{0,\ldots, K-1\}$ as 


$$\mathbf{probs(y = c | x)} = \frac{\exp(\mathbf{w}_c^T\mathbf{x})}{\sum_{k=1}^K \exp(\mathbf{w}_k^T\mathbf{x})} = \mathbf{probs[c]}$$ 

The last layer of the network provides a probability vector $\mathbf{probs} \in \mathbb{R}^K$, such that each $0 \le \mathbf{probs[c]} \le 1$ and $\sum_c \mathbf{probs[c]} = 1$, where $\mathbf{k}$ is the number of classes (10), and $\mathbf{c}$ is a specific class.

For example, if $\mathbf{probs[5]} = 0.42$, that means there is a 42% chance that the image is a 5. For each sample, we assign our image the position of the maximum value in the probs array.

$$\hat{y}_i = \arg \max_c  \mathbf{probs[c]} $$

---

### Load MNIST Dataset 

First, we must load the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) handwritten digits dataset from the Keras package. The dataset consists of 10 handwritten digits (0, 1,..., 9). It's widely used to demonstrate the simple image classification problem.

The training and test data consists of 60000 and 10000 images, respectively, of size $28 \times 28$ pixels. The following code segment plots 10 sample images.

In [ ]:
# Load MNIST Dataset
def loadMNIST():
    from keras.datasets import mnist
    (xTrainMNIST, yTrainMNIST), (xTestMNIST, yTestMNIST) = mnist.load_data()

    # Plot MNIST Examples
    n_img = 10
    plt.figure(figsize = (n_img * 2, 2))
    print("10 Random MNIST Sample Images")
    plt.title("10 Random MNIST Sample Images")
    plt.gray()
    for i in range(n_img):
        plt.subplot(1, n_img, i + 1)
        plt.imshow(xTrainMNIST[np.random.randint(0, xTrainMNIST.shape[0])])
    plt.show()

    # Reshape data 
    xTrainMNIST = xTrainMNIST.reshape(xTrainMNIST.shape[0], -1)
    xTestMNIST = xTestMNIST.reshape(xTestMNIST.shape[0], -1)

    print("Training data shape:", xTrainMNIST.shape, xTrainMNIST.shape[0], "images, 28x28 pixels")
    print("Test data shape:", xTestMNIST.shape, xTestMNIST.shape[0], "images, 28x28 pixels")

    return xTrainMNIST, yTrainMNIST, xTestMNIST, yTestMNIST

xTrainMNIST, yTrainMNIST, xTestMNIST, yTestMNIST = loadMNIST()

---

### Training & Test Data

Here we pick training and test data for multi-class classification. We retrieve 1000 images from each class for a total of $N = 10000$ images and create arrays for input and output. 

We will be vectorizing the training and test images, so the size of each vector will be $784 (28\times 28)$. After transposing the dimension of the data, the training and test dataset sizes become $784\times N$. This will be helpful to feed it to our model based on our notations.

```
# xTrain/xTest -- 784 x N array of training/test input
# yTrain/yTest -- 1 x N array of labels 
```  

In [ ]:
# EXTRACT CLASSIFIED DATASET
def extractAllClassificationDataset(x, y, numSamples):
    # Numpy arrays to store training set
    x_ = np.zeros((0, 784)) # Image Data
    y_ = np.zeros((0)) # Class Labels

    # numSamples samples per label put in numpy arrays
    for label in range(10):
        tempX = x[y == label]
        tempX = tempX[:numSamples]
        tempY = np.full(numSamples, label)
        
        x_ = np.concatenate((x_, tempX), axis = 0)
        y_ = np.concatenate((y_, tempY), axis = 0)

    return x_.T, y_

# SELECT DATA
numSamples = 1000
xTrain, yTrain = extractAllClassificationDataset(xTrainMNIST, yTrainMNIST, numSamples)
xTest, yTest = extractAllClassificationDataset(xTestMNIST, yTestMNIST, numSamples)

---

### Network Architecture

Our 2-layer neural network consists of an input layer with 784 nodes, a hidden layer with 256 nodes and an output layer will have 10 nodes. The first layer will use a ```sigmoid``` activation function and the second layer will use a ```softmax``` activation function.

The equations for feedforward operation will be as follows.

$$\mathbf{z}^{(1)}=W^{(1)} \mathbf{x}+ \mathbf{b}^{(1)}\\\mathbf{y}^{(1)}=\text{sigmoid}(\mathbf{z}^{(1)})\\\mathbf{z}^{(2)}=W^{(2)}  \mathbf{y}^{(1)}+ \mathbf{b}^{(2)} \\\mathbf{probs} = \mathbf{y}^{(2)}=\text{softmax}(\mathbf{z}^{(2)})$$

where $\mathbf{x}\in \mathbb{R}^{784}$ is the input layer, $\mathbf{y}^{(1)}\in \mathbb{R}^{256}$ is the hidden layer, $\mathbf{y}^{(2)} \in \mathbb{R}$ is the output layer, $W^{(1)}\in \mathbb{R}^{256\times 784}$ is the first layer weights, $W^{(2)}\in \mathbb{R}^{10\times 256}$ is the second layer weights, $\mathbf{b}^{(1)}\in \mathbb{R}^{256}$ is the first layer bias, $\mathbf{b}^{(2)}\in \mathbb{R}^{10}$ is the second layer bias vector.

---

### Network initialization

We initialize the weights for $W^{(1)}$ and $W^{(2)}$ with random values drawn from normal distribution with zero mean and 0.01 standard deviation. We will initialize bias vectors $\mathbf{b}^{(1)}$ and $\mathbf{b^{(2)}}$ with zero values. 

We can fix the seed for random initialization for reproducibility.

In [ ]:
# INITIALIZE NEURAL NETWORK
def TwoLayerNetwork(layer_dims=[784, 256, 10]):
    # Initialize Normally Distributed Weights
    W1 = np.random.normal(0, 0.01, (layer_dims[1], layer_dims[0])) # (256, 784)
    W2 = np.random.normal(0, 0.01, (layer_dims[2], layer_dims[1])) # (10, 256)

    # Initialize Biases to 0
    b1 = np.zeros(layer_dims[1]) # (256, )
    b2 = np.zeros(layer_dims[2]) # (10, ))

    # Return array of weights/biases
    params = np.array([W1, b1, W2, b2], dtype=object)
    return params

---

### Sigmoid Activation Function 
The ```sigmoid``` activation function is written as 

$$ \varphi(z) = \frac{1}{1+e^{-z}}$$

Note that derivative of ```sigmoid``` is $\varphi'(z) = \varphi(z) (1-\varphi(z))$. 

In [ ]:
# SIGMOID ACTIVATION FUNCTION
def sigmoid(Z):
    # Input: Z numpy.ndarray
    Y = 1/(1 + np.exp(-Z))
    return Y

---

### Softmax Activation Function
The softmax activation function is a multinomal extension of the sigmoid function that maps a vector of length $K$ to a probability vector.

We can define ```softmax``` function on a vector $\mathbf{z} \in \mathbb{R}^K$ as $\mathbf{p} = \text{softmax}(\mathbf{z})$: 

$$\mathbf{p}_c(\mathbf{z}) = \frac{\exp(\mathbf{z}_c)}{\sum_{k=1}^K \exp(\mathbf{z}_k)}$$



In [ ]:
# SOFTMAX ACTIVATION FUNCTION
def softmax(probs): 
    # probs -- K x N numpy.ndarray, K is the number of classes, N is the number of samples
    probs = np.exp(probs) / np.sum(np.exp(probs), axis = 0)
    return probs

The derivative of the ```softmax``` function with respect to any input can be written as 

$$ \frac{\partial \mathbf{p}_i}{\partial \mathbf{z}_j} = \begin{cases} \mathbf{p}_i(1-\mathbf{p}_j) & i = j \\ \mathbf{p}_i (-\mathbf{p}_j) & i \ne j. \end{cases}$$

---

### Stable Softmax
The numerical range of floating point numbers in numpy is limited. For `float64` the upper bound is $10^{308}$. For exponential, its not difficult to overshoot that limit, in which case python returns `nan`.

To make our softmax function numerically stable, we normalize the values in the vector by multiplying the numerator and denominator with a constant `C` as

\begin{align*}
\mathbf{p}_c  &= \frac{\exp(\mathbf{z}_c)}{\sum_{k=1}^K \exp(\mathbf{z}_k)} \\
& = \frac{C\exp(\mathbf{z}_c)}{C\sum_{k=1}^K \exp(\mathbf{z}_k)}\\
& = \frac{\exp(\mathbf{z}_c + \log C)}{C\sum_{k=1}^K \exp(\mathbf{z}_k + \log C)}.
\end{align*}

We can choose an arbitrary value for `log(C)` term, but generally `log(C) = −max(z)` is chosen


In [ ]:
# STABLE SOFTMAX ACTIVATION FUNCTION
def stable_softmax(probs): 
    # probs -- K x N numpy.ndarray, K is the number of classes, N is the number of samples
    probs = np.exp(probs - np.max(probs)) / np.sum(np.exp(probs - np.max(probs)), axis = 0)
    return probs

---

### Multiclass Cross Entropy Loss Function

We want to minimize the ```cross entropy loss``` using the true labels and predicted labels of a batch of N samples. The multi-class ```cross entropy loss``` for $i^{th}$ sample can be written as

$$Loss_i = -\sum_c \mathbf{1}(y_i = c) \log \mathbf{p}_c $$
where $y_i$ is the true label and 

$$\mathbf{1}(y_i = c) = \begin{cases} 1 & y_i =c \\ 0 & \text{otherwise} \end{cases}$$ 
is an indicator function. 

We can find the average loss for a batch of N samples as $Loss=\frac{1}{N}\sum_{i=1}^{N} Loss_i$. 


In [ ]:
# CROSS ENTROPY LOSS FUNCTION FOR MULTIPLE CLASSES
def MultiClassCrossEntropyLoss(Y_true, probs):
  # probs -- K x N array
  # Y_true -- 1 x N array 
  # loss -- sum Loss_i over N samples 

  N = Y_true.shape[0] # N Samples
  p = stable_softmax(probs)
  log_likelihood = -np.log(p[Y_true.astype(int), range(N)])
  loss = np.sum(log_likelihood)/N
  return loss

---

### Derivative of the Cross Entropy Loss 

Let's assume that $\mathbf{probs} = \text{softmax}(\mathbf{z})$. 

The derivative of the loss w.r.t. $\mathbf{probs}_j$ can be written as 
$$\frac{\partial Loss_i }{\partial \mathbf{probs}_j} = \begin{cases} -1/\mathbf{probs}_j & j = y_i \\ 0 & j \ne y_i \end{cases}. $$

The _total derivative_ is used to compute the derivative of the loss for $i^{th}$ sample w.r.t. $j^{th}$ entry in $\mathbf{z}$ as

\begin{align*}
\frac{\partial Loss_i}{\partial \mathbf{z}_j} = \sum_c \frac{\partial Loss_i}{\partial \mathbf{probs}_c}\frac{\partial \mathbf{probs}_c}{\partial \mathbf{z}_j}.
\end{align*}

From our discussion above, we know that the $\frac{\partial Loss_i}{\partial \mathbf{probs}_c} = 0$ if $c \ne y_i$. 


\begin{align*}
\frac{\partial Loss_i}{\partial \mathbf{z}_j} &= -\frac{1}{\mathbf{probs}_c} \frac{\partial \mathbf{probs}_c}{\partial \mathbf{z}_j} \\
& = \begin{cases} \mathbf{probs}_j - 1 & j = y_i \\ \mathbf{probs}_j & j \ne y_i. \end{cases}
\end{align*}

Therefore, $$\delta^{(2)} = \nabla_{\mathbf{z}^{(2)}} Loss_i = \mathbf{probs} - \mathbf{1}_{y_i}.$$

where $\mathbf{1}_{y_i}$ is a __one-hot vector__ that has length $K$ and is zero everywhere except 1 at index same as $y_i$. 


---

### Forward Propagation 
This is the forward pass for our two layer network. Each layer consists of an affine function (fully-connected layer) followed by an activation function (```sigmoid``` or ```softmax```). 

The forward propagation function returns the intermediate results ($\mathbf{x}, \mathbf{z}^{(1)}, \mathbf{y}^{(1)}, \mathbf{z}^{(2)}$) in addition to the final output ($\mathbf{probs = y}^{(2)}$). The intermediate outputs are used for the backpropagation step.

In [ ]:
# FORWARD PROPAGATION
def forward(X, params):
    # Inputs:
      # X -- 784 x N array 
      # params
        # W1 -- 256 x 784 matrix
        # b1 -- 256 x 1 vector
        # W2 -- 10 x 256 matrix
        # b2 -- 10 x 1 vector 
      
    # Outputs:
      # probs -- 10 x N output

    Z1 = np.matmul(params[0], X) + params[1][:, np.newaxis] # W1X + b1
    Y1 = sigmoid(Z1) # Output of layer 1
    Z2 = np.matmul(params[2], Y1) + params[3][:, np.newaxis] # W2Y1 + b2
    probs = stable_softmax(Z2) # Final output

    # Put all intermediate values in array and return
    intermediate = np.array([X, Z1, Y1, Z2], dtype=object)
    return probs, intermediate

---

### Backpropagration

The backpropagation step for the two layer neural network is implemented using the ```softmax``` layer and ```cross entropy loss``` function. 


The gradient of the Loss w.r.t. $W^{(l)},\mathbf{b}^{(l)}$ for $l = 1,2$ for a single training sample can be written as  

$$\nabla_{W^{(l)}} Loss_i = \delta^{(l)} \mathbf{y}^{(l-1)T},$$  
$$\nabla_{\mathbf{b}^{(l)}} Loss_i = \delta^{(l)},$$


where 
$$\delta^{(l)} = \nabla_{\mathbf{z}^{(l)}} Loss = \nabla_{\mathbf{y}^{(l)}} Loss \odot \varphi'(\mathbf{z}^{(l)}).$$ 

For an $i^{th}$ sample, $\delta^{(2)} = \nabla_{\mathbf{z}^{(2)}} Loss_i = \mathbf{probs} - \mathbf{1}_{y_i},$ where $\mathbf{1}_{y_i}$ is a __one-hot vector__ that has length $K$ and is zero everywhere except 1 at index same as $y_i$, and $\mathbf{probs}$ is the output probability vector for the $i^{th}$ sample. 


Once we have the gradients $\nabla_{W^{(l)}} Loss_i, \nabla_{\mathbf{b}^{(l)}} Loss_i$ for all $i$. We can compute their average to compute the gradient of the total loss function as

$$\nabla_{W^{(l)}} Loss = \frac{1}{N} \sum_i \nabla_{W^{(l)}} Loss_i, $$
$$ \nabla_{\mathbf{b}^{(l)}} Loss = \frac{1}{N} \sum_i  \nabla_{\mathbf{b}^{(l)}} Loss_i.$$

In [ ]:
# BACKPROPAGATION
def backward(Y_true, probs, intermediate, params):
    # Inputs: 
      # Y_true -- 1 x N true labels
      # probs -- 10 x N output of the last layer
      # intermediate -- X, Z1, Y1, Z2 
      # params -- W1, b1, W2, b2 
    
    # Outputs: 
      # grads -- [grad_W1, grad_b1, grad_W2, grad_b2]

    # Num Samples
    N = np.size(Y_true)

    # LAYER 2 (l = L)
    delta2 = np.zeros((10, N))
    Y1 = intermediate[2]

    # Compute delta2
    for i in range(N):
      oneHot = np.zeros(10)
      oneHot[Y_true[i].astype(int)] = 1
      delta2[:, i] = probs[:, i] - oneHot

    # Compute b2GradientAverage/W2GradientAverage
    b2GradientAverage = np.zeros((10))
    for j in range(10):
      b2GradientAverage[j] = np.average(delta2[j, :])
    W2GradientAverage = (1/N) * np.matmul(delta2, Y1.T) # (10, 256)

    # LAYER 1 (l < L)
    W2 = params[2]
    Z1 = intermediate[1]
    
    # Compute delta1
    delta1 = np.matmul(W2.T, delta2) * (sigmoid(Z1) * (1 - sigmoid(Z1))) # (256, 10000)

    # Compute b1GradientAverage
    b1GradientAverage = np.zeros((256))
    for j in range(256):
      b1GradientAverage[j] = np.average(delta1[j, :])

    # Compute W1GradientAverage
    X = intermediate[0]
    W1GradientAverage = (1/N) * np.matmul(delta1, X.T) # (256, 784)

    # Put weight/bias gradients in array and return
    grads = np.array([W1GradientAverage, b1GradientAverage, W2GradientAverage, b2GradientAverage], dtype=object)
    return grads

---

### Gradient Descent Optimizer
We will use a standard gradient descent-based optimizer to minimize the ```cross entropy loss``` function. The learning rate may be adjusted to provide the best training/validation performance. The same learning rate is applied for all weights.

$W^1, \mathbf{b}^1, W^2, \mathbf{b}^2$ are updated as 
$$ W^1 \gets W^1 - \alpha \nabla_{W^1} Loss $$
$$ \mathbf{b}^1 \gets \mathbf{b}^1 - \alpha \nabla_{\mathbf{b}^1} Loss $$ 
$$ W^2 \gets W^2 - \alpha \nabla_{W^2} Loss $$ 
$$ \mathbf{b}^2 \gets \mathbf{b}^2 - \alpha \nabla_{\mathbf{b}^2} Loss $$ 
where $\alpha$ is the learning rate. 

In [ ]:
# GRADIENT DESCENT OPTIMIZER
def GD(params, grads, learning_rate):
    # New params = old params - (learning rate (α) * gradient of Loss computed at old params)
    # params -- W1, b1, W2, b2 
    params[0] = params[0] - (learning_rate * grads[0]) #W1 - αLossW1 (256, 784)
    params[1] = params[1] - (learning_rate * grads[1]) #b1 - αLossb1 (256, )
    params[2] = params[2] - (learning_rate * grads[2]) #W2 - αLossW2 (1, 256)
    params[3] = params[3] - (learning_rate * grads[3]) #b2 - αLossb2 (1, )
    return params

---

### Training the Model
To train our multi-class classificaiton model, we use the forward and backward propagation functions with our gradient descent optimizer. 

First, we specify the number of nodes in the layers, number of epochs, and learning rate.

In [ ]:
# Specify layer dimensions, number of epochs, and learning rate
layerDims = [xTrain.shape[0], 256, 10]
epochs = 250
lr = 0.025

The network is intialized given our layerDims.

In [ ]:
# Initialize Neural Network
params = TwoLayerNetwork(layerDims)

Then, we train the network for the number of epochs specified above. In every epoch, we execute the following:
1. Calculate a forward pass to get estimated labels
2. Use the estimated labels tp calculate and record loss for every epoch
3. Use backpropagation to calculate gradients
4. Use gradient descent to update the weights and biases

The loss value after every epoch is stored in the ```lossHistory``` array and is printed after every 25 epochs


In [ ]:
# Truncate float to n decimal places
def truncate(f, n):
    s = '{}'.format(f)
    if 'e' in s or 'E' in s:
        return '{0:.{1}f}'.format(f, n)
    i, p, d = s.partition('.')
    return '.'.join([i, (d+'0'*n)[:n]])

# True labels (N x 1)
Y_true = yTrain

# Calculate loss for each epoch
print("TRAINING MODEL....")
lossHistory = np.zeros(epochs)
for i in range(epochs):
  # Forward Pass
  probs, intermediate = forward(xTrain, params) # Y2 1 x N output, intermediate[X, Z1, Y1, Z2], params[W1, b1, W2, b2]

  # Backpropagation
  grads = backward(Y_true, probs, intermediate, params)

  # Gradient Descent Update
  params = GD(params, grads, lr)

  # Calculate Loss
  lossHistory[i] = MultiClassCrossEntropyLoss(Y_true, probs)
  if i % 25 == 0: # Print loss every 25 epochs
    print("Loss at", i , "epoch:", truncate(lossHistory[i], 4))


We plot the recorded loss values vs epochs. We can observe that the training loss decreases with the epochs. Optimally, we'd like the loss value to be as close to 0 as possible.

In [ ]:
# Plot Loss vs Epoches
plt.figure()
plt.plot(lossHistory)
print("Epochs vs Training Loss")
plt.title("Epochs vs Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Training Loss")
plt.show()

---

### Accuracy Evaluation
Now that the network is finished training, we evaluate the accuracy from the trained model. We feed training data and test data to the forward model along with our new trained parameters. 

To convert the probability output of the forward pass into labels, we can assign the label based on the maximum probability. 

$$\hat{y}_i = \arg \max_c  \mathbf{probs[c]} $$

It is always possible to increase accuracy by reducing our Loss further with more epochs or an adjusted learning rate.

In [ ]:
# Compute the training/test accuracy and return correct/wrong indexes
def computeAccuracy(Y_true, probs):
    N = probs.shape[1]
    correct = 0
    correctIndexes = np.zeros(0)
    wrongIndexes = np.zeros(0)
    classifications = np.zeros(0)

    for i in range(N):
        classifications = np.append(classifications, np.argmax(probs[:, i]))
        if(classifications[i] == Y_true[i]):
            correct += 1
            correctIndexes = np.append(correctIndexes, i)
        else:
            wrongIndexes = np.append(wrongIndexes, i)
    
    accuracy = (correct/N) * 100
    return accuracy, classifications, correctIndexes.astype(np.int64), wrongIndexes.astype(np.int64)

# TRAINING ACCURACY
trainingAccuracy, trainingClassifications, correctTrainingIndexes, wrongTrainingIndexes = computeAccuracy(Y_true, probs)
print("TRAINING ACCURACY:", truncate(trainingAccuracy, 4), "%")

# TEST ACCURACY
# Use optimized parameters on a forward pass of the test data set
probs_test = forward(xTest, params)[0]
testAccuracy, testClassifications, correctTestIndexes, wrongTestIndexes = computeAccuracy(yTest, probs_test)
print("TEST ACCURACY:", truncate(testAccuracy, 4), "%")

---

### Visualization of Correct/Miscalassified Images

Next, we look at some images from training and test sets that were correctly or incorrectly classified. We can observe that incorrectly classified images could look very distinct compared to their correctly classified counterparts.

In [ ]:
# Plot correctly & wrongly classified images from the dataset
def plotClassifiedImages(n_img, correctIndexes, wrongIndexes, dataset, typeString):
    newDataset = dataset.reshape(28, 28, dataset.shape[1])
    totalRows = int(math.ceil(n_img / 10))

    # Correctly Classified
    print(n_img, "correctly classified images from", typeString, "dataset")
    maxImages = bool(False)
    displayedImages = np.zeros(0)
    for row in range(totalRows): # For each row
        imagesOnRow = min(10, n_img - (row * 10)) # 10 max images per row
        plt.figure(figsize = (imagesOnRow * 2, 2))
        plt.gray()

        # Plot imagesOnRow images
        for i in range(imagesOnRow):
            plt.subplot(1, imagesOnRow, i + 1)
            imageIndex = correctIndexes[np.random.randint(0, correctIndexes.size)]

            # Don't plot images that have already been displayed
            while(imageIndex in displayedImages):
                imageIndex = correctIndexes[np.random.randint(0, correctIndexes.size)]
            displayedImages = np.append(displayedImages, imageIndex)

            # Random correctly classified image
            image = newDataset[:, :, imageIndex]
            plt.imshow(image)

            # No more images left
            if (displayedImages.size == correctIndexes.size):
                maxImages = bool(True)
                break

        plt.show()
        if(maxImages):
            print("Maximum images displayed:", wrongIndexes.size)
            break
        row += 1

    # Wrongly Classified
    print(n_img, "wrongly classified images from", typeString, "dataset")
    maxImages = bool(False)
    displayedImages = np.zeros(0)
    for row in range(totalRows): # For each row
        imagesOnRow = min(10, n_img - (row * 10)) # 10 max images per row
        plt.figure(figsize = (imagesOnRow * 2, 2))
        plt.gray()

        # Plot imagesOnRow images
        for i in range(imagesOnRow):
            plt.subplot(1, imagesOnRow, i + 1)
            imageIndex = wrongIndexes[np.random.randint(0, wrongIndexes.size)]

            # Don't plot images that have already been displayed
            while(imageIndex in displayedImages):
                imageIndex = wrongIndexes[np.random.randint(0, wrongIndexes.size)]
            displayedImages = np.append(displayedImages, imageIndex)

            # Random wrongly classified image
            image = newDataset[:, :, imageIndex]
            plt.imshow(image)

            # No more images left
            if (displayedImages.size == wrongIndexes.size):
                maxImages = bool(True)
                break
            
        plt.show()
        if(maxImages):
            print("Maximum images displayed:", wrongIndexes.size)
            break
        row += 1

# Plot five correctly & wrongly classified images from the training and test sets
plotClassifiedImages(5, correctTrainingIndexes, wrongTrainingIndexes, xTrain, "training")
plotClassifiedImages(5, correctTestIndexes, wrongTestIndexes, xTest, "test")

---

### Visualize More Images
Finally, the program prompts the user if they'd like to view more correctly/incorrectly classified images. The user can choose which dataset they'd like to view images from, how many images they'd like to display, and whether they'd like to view images from a specific class (digit).

In [ ]:
# View more correctly & wrongly classified images from either dataset
while(True):
    choice = input("Would you like to view more images? (y/n) ")
    if choice.lower() == "y":
        
        # Select training or test dataset
        trainingOrTest = input("Which dataset? (training/test) ")
        while(trainingOrTest.lower() != "training" and trainingOrTest.lower() != "test"):
            trainingOrTest = input("Select a valid dataset (training/test) ")

        # Select number of images to plot
        numImages = int(input("How many images would you like to display? "))
        while(numImages < 0 or numImages > 200):
            numImages = int(input("Select a valid number of images (Max = 200) "))

        # Select specific number to plot
        numberChoice = input("Would you like to plot a specific number? (y/n) ")
        while(numberChoice.lower() != "y" and numberChoice.lower() != "n"):
            numberChoice = input("Select a valid choice (y/n) ")

        # Initialization for arrays to use in plotClassifiedImages()
        dataset = np.zeros((784, 0))
        correctIndexes = np.zeros(0)
        wrongIndexes = np.zeros(0)

        # Build new dataset and indexes for 1 number
        if numberChoice.lower() == "y":
            number = int(input("Which number would you like to plot? (0-9) "))
            while(number < 0 or number > 9):
                number = int(input("Select a valid number (0-9) "))

            # Build from training dataset
            if trainingOrTest.lower() == "training":
                currentIndex = 0
                for i in range(correctTrainingIndexes.size):
                    if(yTrain[correctTrainingIndexes[i]] == number):
                        correctIndexes = np.append(correctIndexes, currentIndex)
                        dataset = np.append(dataset, np.vstack(xTrain[:, correctTrainingIndexes[i]]), 1)
                        currentIndex += 1
                for i in range(wrongTrainingIndexes.size):
                    if(yTrain[wrongTrainingIndexes[i]] == number):
                        wrongIndexes = np.append(wrongIndexes, currentIndex)
                        dataset = np.append(dataset, np.vstack(xTrain[:, wrongTrainingIndexes[i]]), 1)
                        currentIndex += 1
                
            # Build from test dataset
            elif trainingOrTest.lower() == "test":
                currentIndex = 0
                for i in range(correctTestIndexes.size):
                    if(yTest[correctTestIndexes[i]] == number):
                        correctIndexes = np.append(correctIndexes, currentIndex)
                        dataset = np.append(dataset, np.vstack(xTest[:, correctTestIndexes[i]]), 1)
                        currentIndex += 1
                for i in range(wrongTestIndexes.size):
                    if(yTest[wrongTestIndexes[i]] == number):
                        wrongIndexes = np.append(wrongIndexes, currentIndex)
                        dataset = np.append(dataset, np.vstack(xTest[:, wrongTestIndexes[i]]), 1)
                        currentIndex += 1
            
        # Use entire training/test dataset
        else: 
            if trainingOrTest.lower() == "training":
                dataset = xTrain
                correctIndexes, wrongIndexes = correctTrainingIndexes, wrongTrainingIndexes
            elif trainingOrTest.lower() == "test":
                dataset = xTest
                correctIndexes, wrongIndexes = correctTestIndexes, wrongTestIndexes

        # PLOT
        plotClassifiedImages(numImages, correctIndexes.astype(np.int64), wrongIndexes.astype(np.int64), dataset, trainingOrTest.lower())
    else:
        print("Exiting program....")
        break
